# Refinitiv Data Library for Python
## Content - Pricing - Synchronous calls

This notebook demonstrates how to use the Pricing interface to retrieve snapshot data from the Refinitiv Data Platform.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [14]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [15]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [16]:
# Open the Default Session
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x1c97edbeaf0 {session_name='rdp'}>

## Get Price Snapshot
The following gets a snapshot from the non-streaming RDP Pricing API - note the **get_data()** call

In [17]:
# Define our RDP Snapshot Price object
response = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_data()

display(response.data.df)

,Instruments,BID,ASK
0,EUR=,1.1315,1.1317
1,GBP=,1.3289,1.3294
2,JPY=,113.41,113.42
3,CAD=,1.2809,1.2813


## Alternative method for Snapshot data
If the above code generates an error, this could be because you are not licensed for the RDP Snapshot API OR you are using a **Deployed** session.   
If this is the case, then the following can be used to request a Snapshot from the Streaming data feed (assuming you have a Streaming Data licence) - note the **get_stream()** call

In [8]:
# Define our Streaming Price object
non_streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# We want to just snap the current prices, don't need updates
# Open the instrument in non-streaming mode
non_streaming.open(with_updates=False)
# Snapshot the prices at the time of the open() call
non_streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1314,1.1318
1,GBP=,1.329,1.3294
2,JPY=,113.51,113.53
3,CAD=,1.2805,1.2806


The non-streaming requests are closed by the server - **no need to close them**.   

### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [9]:
non_streaming['EUR=']['BID']

1.1314

In [10]:
eur = non_streaming['EUR=']
eur['ASK']

1.1318

#### Iterate on fields

In [11]:
print('JPY=')
for field_name, field_value in non_streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 113.51
	ASK : 113.53


#### Iterate on instruments and fields

In [12]:
for instrument in non_streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.1314
	ASK : 1.1318
GBP=
	BID : 1.329
	ASK : 1.3294
JPY=
	BID : 113.51
	ASK : 113.53
CAD=
	BID : 1.2805
	ASK : 1.2806


## Close the session

In [13]:
rd.close_session()